<a href="https://colab.research.google.com/github/Chinmaysahoo03/RAG_Document_QA/blob/main/RAG_Document_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-groq langchain sentence-transformers faiss-cpu pypdf
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
from google.colab import userdata, files
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.llms import HuggingFacePipeline
from langchain.schema import Document
import warnings
warnings.filterwarnings("ignore")

# Groq Setup
groq_api = userdata.get('groq_api')
llm = ChatGroq(api_key=groq_api, model="gemma2-9b-it", temperature=0.1)

# Embeddings (free Hugging Face model)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("Setup complete!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setup complete!


In [3]:
# Function to load docs (PDF or text)
def load_documents(file_path):
    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    else:
        loader = TextLoader(file_path)
    docs = loader.load()
    return docs

# Sample: Upload a file (run this to upload)
uploaded = files.upload()  # Upload a PDF or TXT (e.g., INFOSYS -APTITUDE-MODEL PAPERS_1473177928759.pdf)
file_name = list(uploaded.keys())[0]  # Get first uploaded file
docs = load_documents(file_name)
print(f"Loaded {len(docs)} documents.")

# Split into chunks and clean
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Filter out empty or invalid chunks
cleaned_splits = [doc for doc in splits if doc.page_content and doc.page_content.strip()]
print(f"Split into {len(cleaned_splits)} valid chunks after cleaning.")

Saving Gharpayy PRODUCT Plan Chinmaya Sahoo.pdf to Gharpayy PRODUCT Plan Chinmaya Sahoo.pdf
Loaded 4 documents.
Split into 10 valid chunks after cleaning.


In [4]:
# Create FAISS vector store
vectorstore = FAISS.from_documents(splits, embeddings)

# Save index (optional, for reuse)
vectorstore.save_local("faiss_index")

print("Vector store built! Ready for retrieval.")

Vector store built! Ready for retrieval.


In [5]:
# Prompt template for Q&A with citations
prompt_template = """
Use the following context to answer the question. If you don't know, say so. Cite sources with [source chunk index].

Context: {context}

Question: {question}

Answer:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Retrieval QA Chain (top-3 chunks)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

print("RAG chain ready!")

RAG chain ready!


In [6]:
# Interactive loop
print("RAG Q&A System Ready! Type 'quit' to exit.")

while True:
    query = input("\nQuestion: ")
    if query.lower() == 'quit':
        break

    # Run query
    result = qa_chain({"query": query})
    answer = result['result']
    sources = result['source_documents']

    print(f"\nAnswer: {answer}")
    print("\nSources:")
    for i, doc in enumerate(sources):
        print(f"[Chunk {i+1}]: {doc.page_content[:200]}... (Page: {getattr(doc.metadata, 'page', 'N/A')})")

RAG Q&A System Ready! Type 'quit' to exit.

Question: give an overview of the pdf

Answer: This document outlines a plan for developing an Android app focused on tenant search and PG (Paying Guest) management. 

**Key Features:**

* **Tenant Search:**  Users can search for suitable PG accommodations.
* **PG Details:**  Comprehensive information about available PGs.
* **Booking:**  A streamlined booking process for tenants.
* **Admin Features:** Basic administrative functionalities for managing the platform.

**Development Process:**

* **Sprint-Based:** The project is structured into sprints with specific tasks and deadlines.
* **Tools:**  Android Studio, Figma, Firebase, GitHub, and Postman are utilized for coding, design, backend integration, version control, and API testing.
* **AI Assistance:** GitHub Copilot is employed for code suggestions and debugging.
* **Testing:** JUnit is used for unit testing, and Firebase App Distribution handles test APK distribution.

**Deliverables:**
